# McDonalds Diet Problem

In this notebook, we will determine the minimum cost McDonalds diet

In [ ]:
foods = [:QP, :MD, :BM, :FF, :MC, :FR, :SM, :M1, :OJ]
# Word to the wise---Julia symbols cannot start with a number (M1 instead of 1M. It took me a while to figure this out)

nutrients = [:Prot, :VitA, :VitC, :Calc, :Iron, :Cals, :Carb]

cost = Dict(zip(foods,[1.84,2.19,1.84,1.44,2.29,0.77,1.29,0.6,0.72]))
required = Dict(zip(nutrients,[55,100,100,100,100,2000,350]))
using NamedArrays
A = [
    28 24 25 14 31 3 15 9 1
    15 15 6 2 8 0 4 10 2
    6 10 2 0 15 15 0 4 120
    30 20 25 15 15 0 20 30 2
    20 20 20 10 8 2 15 0 2
    510 370 500 370 400 220 345 110 80
    34 33 42 38 42 26 27 12 20
]
A_NA = NamedArray(A, (nutrients,foods), ("Nutrients","Menu Items")) ;





In [ ]:
burgers = [:QP, :MD, :BM] ;

Suppose instead of minimizing cost, I wish to maximize the number of hamburgers I eat.  (I like hamburgers A LOT!)

In [ ]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
@variable(m, x[foods] >= 0)
@objective(m, Max, sum(x[j] for j in burgers))
@constraint(m, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])

# If you don't want the solver output, you can make it slient like this
#set_silent(m)
optimize!(m)

# You can use the Julia Formatting library for prettier printing, specifying precision, or you can use @printf like C
using Formatting
printfmtln("Maximum Number Hamburgers {:.2f}: ", objective_value(m))
for j in foods
    if value(x[j]) > 1.0e-6
        printfmtln("Eat {1:.2f} of menu item {2}", value(x[j]), j)
    end
end



See that the Model status is Unbounded!
In this case the "solution" returned is garbage.
We need to make sure the Model Status is 'Optimal' in order to 'trust' the solution.
See https://jump.dev/JuMP.jl/stable/manual/solutions/ for manual page

In [ ]:
opt_status = termination_status(m)
if opt_status != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", stat)
end


Here DUAL_INFEASIBLE status means that the LP instance was UNBOUNDED.
(We will learn more about duality later in the class)

In the interest of extended my life, my wife has requested that I obey the following constraints:

1. Don't eat more than 3 sandwiches per day
2. Don't drink too much  (Always very good advice).  At most 3 drinks per day.
3. Don't eat more than 2 orders of french fries per day.


In [ ]:
sammiches = [:QP, :MD, :BM, :FF, :MC, :SM]

# Let's just make a new model for this
m2 = Model(HiGHS.Optimizer)
@variable(m2, x[foods] >= 0)
@objective(m2, Min, sum(cost[j]*x[j] for j in foods))
@constraint(m2, [i in nutrients], sum(A_NA[i,j]*x[j] for j in foods) >= required[i])
# We can 'name' the constraints if we wish
@constraint(m2, MaxSammich, sum(x[j] for j in sammiches) <= 3)
@constraint(m2, MaxDrinks, x[:M1] + x[:OJ] <= 3)
@constraint(m2, MaxFF, x[:FF] <= 2)
optimize!(m2)

opt_status = termination_status(m2)
if opt_status != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", stat)
end

* Suppose I think that the ``too much drinking'' constraint is the one causing the problem to be infeasible

* Let's create a new problem to determine if I had more than 3 drinks, the instance could be feasible 

* $s$:  Number of extra drinks (over three)
that I must drink in order to get a feasible solution
$$x_{1M} + x_{OJ} - s \leq 3, s \geq 0$$
* New Objective: $\min s$

In [ ]:
using Printf
# We can delete constraints from a model 
delete(m2,MaxDrinks)
# Add a new variable
@variable(m2, s >= 0)
@constraint(m2, NewMaxDrinks, x[:M1] + x[:OJ] - s <= 3)
# This will over-write the old objective in model m2
@objective(m2, Min, s)

optimize!(m2)
opt_status = termination_status(m2)
if stat != MOI.OPTIMAL
    println("Solver did not find optimal solution, status: ", stat)
else
    @printf("I have to drink %.2f (extra) drinks\n", objective_value(m2))
    @printf("%.2f milk, %.2f OJ", value(x[:M1]), value(x[:OJ]))
end